In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import os.path
from astropy.coordinates import Angle
from astropy import units as u
from scipy.stats import norm

<hr/>

**INSTRUCTION** <br/>
**Sections**: <br/>
-Extracting RA and DEC, and Redshift <br/>
-Plotting <br/>
-HI and CO Maps and Histograms 

**-Extracting RA and DEC, and Redshift** <br/>
1.Create a folder titled "Redshift CSV" in the same workspace as this notebook. Your CSV file contining the RA, Dec, Redshift, and QOP will be stored here.<br/>
2.Below the "Saving DataFrame as CSV File" is the #BREAK. Run everything from there and above. <br/>
3.This will find the RA, Dec, Redshift, and QOP -> Find if there an existing CSV file to save them  -> If no, create the appropriate file to save it -> If yes, open it and add to it. <br/>
4.Change mask_name to a new file and run for all mask. <br/>

**-Plotting** <br/>
1."Plotting" and "Using Delta_RA and Delta_Dec" sections will produce your velocity map. Run these two sections after you have run all masks through the previous section. <br/>
2.Four plots will be produced. Two maps will be your velocity maps with Dec and RA. Two maps will be your velocity maps with Delta Dec and Delta RA. 

**-HI and CO Maps** <br/>
1.Required M33_2018b_phot_spec.txt to run <br/>
2.Run section to extract data, convert to deg, and plot. <br/>

**-H$\alpha$ - HI, H$\alpha$ - CO, HI - CO** <br/>
1.Run section as it is.

**-Testing For Drags** <br/>
1.Run section as it is.
<hr/>

## Variables Pannel

In [ ]:
mask_name = "E2M33R" #Input mask name

## Get Excluded and Includes Slit Numbers

In [ ]:
def get_exclusions():
    filepath = 'ISM_EM_LINES.txt'
    fp = open(filepath)
    all_data = []
    for line in (fp):
        mask_name = line.split(':')[0].split('_')[0]
        slit_number = line.split(':')[1].strip().split(" ")[0]
        if len(slit_number) == 2:
            slit_number = '0' + slit_number
        elif len(slit_number) == 1:
            slit_number = '00' + slit_number
        else:
            pass
        object_id = line.split(':')[1].strip().split()[1]
        data = {}
        data['mask_name'] = mask_name
        data['slit_number'] = slit_number
        data['object_id'] = object_id
        all_data.append(data)
    return all_data     

In [ ]:
def get_files_to_include(folder):
    import os
    list_of_files_to_include = []
    list_of_files_to_exclude = []
    serendip_files = []
    all_file_names_in_folder = os.listdir('data/{}'.format(folder))
    y = len(all_file_names_in_folder)
    print("The number of files in the folder is {0}".format(y))
    all_data = get_exclusions()
    len_all_data = len(all_data)
    for n in range(y):
        parts_of_file_name = all_file_names_in_folder[n].split(".")
        if parts_of_file_name[0] == 'spec1d': # avoids hidden DS_Store files on my mac
            object_id = parts_of_file_name[3]
            slit_number = parts_of_file_name[2]
            mask_name = parts_of_file_name[1]
            should_include = True
            should_exclude = True
            for k in range(len_all_data):
                if ((object_id == all_data[k]['object_id']) and (slit_number == all_data[k]['slit_number']) and (mask_name == all_data[k]['mask_name'])):
                    should_include = False
                    should_exclude = True
                if 'serendip' in object_id:
                    should_include = False
                    should_exclude = False
            if should_include == True:
                list_of_files_to_include.append(all_file_names_in_folder[n])       
            elif should_exclude == True:
                list_of_files_to_exclude.append(all_file_names_in_folder[n])
            elif should_include == False & should_exclude == False:
                serendip_files.append(all_file_names_in_folder[n])
    
    print('The number of files left after exclusions is {0}'.format(len(list_of_files_to_include)))
    
    return sorted(list_of_files_to_include), sorted(list_of_files_to_exclude), sorted(serendip_files)


In [ ]:
def get_slit_nums(files):
    
    slit_nums = []
    
    if len(files) > 1:
    
        for i in range(len(files)):
            parts_of_file_name = files[i].split(".")
            slit_num = parts_of_file_name[2]
            slit_nums.append(int(slit_num))
            
    return slit_nums

In [ ]:
#filtering files
list_of_files_to_include, list_of_files_to_exclude, list_of_serendip_files = get_files_to_include(mask_name)

file_names = list_of_files_to_include #included
file_names_exclude = list_of_files_to_exclude #excluded
file_names_serendip = list_of_serendip_files #serendips

In [ ]:
slit_nums = get_slit_nums(file_names)
slit_nums_exclude = get_slit_nums(file_names_exclude)

print("Slit # to INCLUDE in median calculation: {0}".format(slit_nums))
print("Slit # to EXCLUDE: {0}".format(slit_nums_exclude))
print("Len of EXCL Slits: {0}".format(len(slit_nums_exclude)))

## Read RA, DEC, and Object ID

In [ ]:
def read_sky_coor(mask_name,list_of_excl_files,slit_nums_exclude):
    
    ra = [] #in deg
    dec = [] #in deg
    object_id = [] #object id
    
    for file in list_of_excl_files:
        #extracting RA and DEC from data files
        open_fits = fits.open("./data/{0}/{1}".format(mask_name,file))
        ra_obj = open_fits[1].header["RA_OBJ"].split(":") #[Hour,Min,Sec]
        dec_obj = open_fits[1].header["DEC_OBJ"].strip("+").split(":") #[Deg,Min,Sec]
        
        try: #save OBJID as int. If OBJID has letter, save as str
            obj_id = int(open_fits[1].header["OBJNO"])
        except:
            obj_id = open_fits[1].header["OBJNO"]
        
        #converting RA (HMS) to degrees format
        ra_obj_deg = (float(ra_obj[0]) + (float(ra_obj[1])/60) + (float(ra_obj[2])/3600)) * 15
        ra.append(ra_obj_deg)
        
        #converting Dec (DMS) to degrees format
        dec_obj_deg = (float(dec_obj[0]) + (float(dec_obj[1])/60) + (float(dec_obj[2])/3600))
        dec.append(dec_obj_deg)
    
        #add Object ID to list
        object_id.append(obj_id)
        
    #create DataFrame
    df = pd.DataFrame({"Mask Name":np.full(len(ra),mask_name),"Slit Number":slit_nums_exclude,
                       "RA":ra,"DEC":dec,"OBJECT ID":object_id})
    
    return df

In [ ]:
df = read_sky_coor(mask_name,file_names_exclude,slit_nums_exclude)

## Extract Redshift

In [ ]:
import numpy
def loadMarzResults(filepath):
    return numpy.genfromtxt(filepath, delimiter=',', skip_header=2, autostrip=True, names=True, dtype=None)

In [ ]:
#arrays containing all slits processed in MARZ
res = loadMarzResults("./Marz_Results (Redone)/{0}_Marz_KN.mz".format(mask_name)) 
confident = res[res['QOP'] == 4]

In [ ]:
redshift = {} #empty dict for redshift value
QOP = {} #empty dict for QOP

for n in range(len(res)): #sorting 
    redshift["Slit_{0}".format(slit_nums_exclude[n])] = res[n][12]*300000 #km/s
    QOP["Slit_{0}".format(slit_nums_exclude[n])] = res[n][13]

df["Velocity (km/s)"] = redshift.values() #adding column titled "Velocity" into DF
df["QOP"] = QOP.values() #adding column titled "QOP" into DF

## Saving DataFrame as a CSV file

In [ ]:
def savingdf_csv(df):
    if os.path.isfile("./Redshift CSV/Redshift.csv") == False: #isfile check if a file exist
        df.to_csv("./Redshift CSV/Redshift.csv",index=False)
    else:
        csv_to_df = pd.read_csv("./Redshift CSV/Redshift.csv")
        new_df = pd.concat([csv_to_df,df]) #concat add two DF together 
        new_df.to_csv("./Redshift CSV/Redshift.csv",index=False) #put index_column = False so that 
        #the overwritten csv file don't have extra column called "unnamed"

In [ ]:
savingdf_csv(df)

In [ ]:
#BREAK

In [ ]:
#BREAK

In [ ]:
#BREAK

## Plotting

In [ ]:
M33_Center_RA = (1 + (33/60) + (50.8966/3600))*15 #in deg
M33_Center_Dec = (30 + (39/60) + (36.630/3600)) # in deg

In [ ]:
df1 = pd.read_csv("./Redshift CSV/Redshift.csv")
df1_qop2_sorted = df1["QOP"] >= 2 #use only values that has a QOP of >= 2
df1_qop3_sorted = df1["QOP"] == 3 #use only values that has a QOP of 3

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("M33 Velocity Color Map (QOP >= 2)",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
ax.set_aspect(1/0.861)
cc_map = ax.scatter(df1["RA"][df1_qop2_sorted]
                    ,df1["DEC"][df1_qop2_sorted]
                    ,c=df1["Velocity (km/s)"][df1_qop2_sorted],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
ax.scatter(M33_Center_RA,M33_Center_Dec,marker="s",color="green",s=40,label="Center")
plt.legend()
plt.grid(True,linestyle="--",linewidth=0.7)
fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("Velocity Map (QOP >= 2)")

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("M33 Velocity Color Map (QOP == 3)",fontsize=14)
ax.set_aspect(1/0.861)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
cc_map = ax.scatter(df1["RA"][df1_qop3_sorted]
                    ,df1["DEC"][df1_qop3_sorted]
                    ,c=df1["Velocity (km/s)"][df1_qop3_sorted],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
ax.scatter(M33_Center_RA,M33_Center_Dec,marker="s",color="green",s=40,label="Center")
plt.legend()
plt.grid(True,linestyle="--",linewidth=0.7)
fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("Velocity Map")

## Using Delta_RA and Delta_Dec

In [ ]:
#ra_m32 = 01:33:50.8966
#dec_m32 = +30:39:36.630
ra_m32 = (1 + (33/60) + (50.8966/3600))*15 #in deg
dec_m32 = (30 + (39/60) + (36.630/3600)) # in deg

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Delta RA (deg)",fontsize=12)
ax.set_ylabel("Delta Dec (deg)",fontsize=12)
ax.set_title("M33 Velocity Color Map (QOP >= 2)",fontsize=14)
ax.invert_xaxis()
ax.set_aspect("equal")
cc_map = ax.scatter(((df1["RA"][df1_qop2_sorted]-ra_m32)*np.cos(dec_m32))
                    ,(df1["DEC"][df1_qop2_sorted]-dec_m32)
                    ,c=df1["Velocity (km/s)"][df1_qop2_sorted],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
plt.grid(True,linestyle="--",linewidth=0.7)
fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("Velocity Map (QOP >= 2) w Del RA and Dec")

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Delta RA (deg)",fontsize=12)
ax.set_ylabel("Delta Dec (deg)",fontsize=12)
ax.set_title("M33 Velocity Color Map (QOP == 3)",fontsize=14)
ax.invert_xaxis()
ax.set_aspect("equal")
cc_map = ax.scatter(((df1["RA"][df1_qop3_sorted]-ra_m32)*np.cos(dec_m32))
                    ,(df1["DEC"][df1_qop3_sorted]-dec_m32)
                    ,c=df1["Velocity (km/s)"][df1_qop3_sorted],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
plt.grid(True,linestyle="--",linewidth=0.7)
fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("Velocity Map w Del RA and Dec")

In [ ]:
#BREAK

In [ ]:
#BREAK

In [ ]:
#BREAK

## HI and CO Maps and Histograms

In [ ]:
def read_M33_phot_spec():
    
    """
    Parameters
    ----------
    None
    
    Returns
    ----------
    RA: 1d-array containing RA in unit of deg.
    Dec: 1d-array containing Dec in unit of deg.
    maskname: list containing name of mask in M33_2018b_phot_spec.txt
    HI: 1d-array containing velocity measurements in HI region.
    CO: 1d-array containing velocity measurements in CO region.
    
    """
    
    RA = [] #Empty list to append RA
    Dec = [] #Empty list to append Dec
    maskname = [] #Empty list to append mask name
    HI = [] #Empty list to append v of HI region
    CO = [] #Empty list to append v of CO region
    OBJID = []
    
    M33_phot_spec_lines = open("M33_2018b_phot_spec.txt","r").readlines()
    for line in M33_phot_spec_lines[1:]: #[1:0] so that it does NOT read the labels
        RA.append(Angle(line.split()[1],u.hourangle).degree) #converting RA to Deg
        Dec.append(Angle(line.split()[2],u.deg).degree) #converting Dec to Deg
        maskname.append(line.split()[16]) #mask name of current line
        HI.append(float(line.split()[19])) #HI measurement of current line
        CO.append(float(line.split()[20])) #CO measurement of current line
        OBJID.append(line.split()[0]) #OBJID 
        
    return np.array(RA),np.array(Dec),maskname,np.array(HI),np.array(CO),OBJID
        

In [ ]:
Ri_As,Decli,Maskname,HI,CO,OBJID = read_M33_phot_spec()

In [ ]:
HI_index = np.isnan(HI)==False #remove nan values from HI
CO_index = np.isnan(CO)==False #remove nan values from CO

### Plotting HI and CO Maps

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("HI Region",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
ax.set_aspect(1/0.861)
cc_map = ax.scatter(Ri_As[HI_index],Decli[HI_index],c=HI[HI_index],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
ax.scatter(M33_Center_RA,M33_Center_Dec,marker="s",color="green",s=40,label="Center")
plt.legend()
fig.set_figwidth(5)
fig.set_figheight(5)
plt.grid(True,linestyle="--",linewidth=0.7)
plt.savefig("HI Region")
plt.show()

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("CO Region",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
ax.set_aspect(1/0.861)
cc_map = ax.scatter(Ri_As[CO_index],Decli[CO_index],c=CO[CO_index],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
ax.scatter(M33_Center_RA,M33_Center_Dec,marker="s",color="green",s=40,label="Center")
plt.legend()
plt.grid(True,linestyle="--",linewidth=0.7)
fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("CO Region")
plt.show()


In [ ]:
#BREAK

In [ ]:
#BREAK

In [ ]:
#BREAK

## H$\alpha$ - HI, H$\alpha$ - CO, HI - CO

### Determine # of Shared OBJ and Make Plots

In [ ]:
def shared_obj(Maskname,OBJID):
    """
    Parameters
    ----------
    Maskname: list, req
        List containing mask names of all mask in M33_2018b_phot_spec.txt
    OBJID: list, req
        List containing Object ID of all objects in M33_2018b_phot_spec.txt
    
    Returns
    ----------
    Bool: List containing booleans of Redshift.csv objects in M33_2018b_phot_spec.txt
    Bool_Index: List containing indices of True values in Bool.
    csv_data: DataFrame containing content in Redshift.csv
    """
    csv_data = pd.read_csv("./Redshift CSV/Redshift.csv") #Read Redshift.csv and extract the Mask Namd and OBJID columns
    OBJID_myData = csv_data["OBJECT ID"]
    Maskname_myData = csv_data["Mask Name"]
    TupPairs_myData = list(zip(Maskname_myData,OBJID_myData)) #Create pairs of data
    TupPairs = list(zip(Maskname,OBJID)) #Create pairs of data from M33_2018b_phot_spec.txt
    Bool = [pair in TupPairs for pair in TupPairs_myData] #List of Bool checking whether objects in Redshift.csv is in M33_2018b_phot_spec.txt
    Bool_Index = [TupPairs.index(TupPairs_myData[index]) for index in range(len(Bool)) if Bool[index] == True]
    #List of Bool checking whether objects in M33_2018b_phot_spec.txt is in Redshift.csv
    
    return Bool,Bool_Index,csv_data


In [ ]:
Bool,Bool_Index,csv_data = shared_obj(Maskname,OBJID)
myDataDF = pd.DataFrame({"Mask Name":csv_data["Mask Name"][np.where(np.array(Bool)==True)[0]],
                        "OBJID":csv_data["OBJECT ID"][np.where(np.array(Bool)==True)[0]],
                        "Velocity (km/s)":csv_data["Velocity (km/s)"][np.where(np.array(Bool)==True)[0]],
                        "Mask Name(Check)":np.array(Maskname)[np.array(Bool_Index)],
                        "OBJID(Check)":np.array(OBJID)[np.array(Bool_Index)],
                        "HI":HI[np.array(Bool_Index)],
                        "CO":CO[np.array(Bool_Index)],
                        "RA":Ri_As[np.array(Bool_Index)],
                        "Dec":Decli[np.array(Bool_Index)]}) #DF to stores relevant info
 
myDataDF.to_csv("./Redshift CSV/Redshift (incl. CO and HI).csv",index=False)
                        

In [ ]:
myDataDF

### Create Histograms of HAlpha-HI, HAlpha-CO, and HI-CO

In [ ]:
myDataDF = pd.read_csv("./Redshift CSV/Redshift (incl. CO and HI).csv")
V = np.array(myDataDF["Velocity (km/s)"]) #velocity calculated by me
HI_V = np.array(myDataDF["HI"]) #HI velocity from Amanda's data
CO_V = np.array(myDataDF["CO"]) #CO velocity from Amanda's data
VsubHI = V - HI_V #Halpha - HI
VsubCO = V - CO_V #Halpha - CO
HIsubCO = HI_V - CO_V #HI - CO

In [ ]:
def normal_distribution(x,mu,sigma):
    return (1/(np.sqrt(2*np.pi)*sigma))*np.exp((-1/(2*(sigma**2)))*((x-mu)**2))

In [ ]:
x = np.linspace(-250,250,1000)
#mu, sigma = norm.fit(VsubHI[~np.isnan(VsubHI)]) #estimate mean and standard dev
#f_x = normal_distribution(x,mu,sigma) #normal distribution formula
#p = norm.pdf(filler_data,mu,std)
median = np.median(VsubHI[~np.isnan(VsubHI)])
width = np.std(VsubHI[~np.isnan(VsubHI)])
Hist_info = plt.hist(VsubHI[~np.isnan(VsubHI)],bins=50)
#plt.plot(x,f_x,label="Gaussian Curve")
plt.title("Peak={0},Width={1}".format(round(median,2),round(width,2)))
plt.xlabel("H-Alpha Velocity - HI Velocity")
plt.xlim(-100,100)
plt.savefig("HAlpha-HI Histogram.png")

In [ ]:
#mu1, sigma1 = norm.fit(VsubCO[~np.isnan(VsubCO)])
#f_x1 = normal_distribution(x,mu1,sigma1) #normal distribution formula
#p1 = norm.pdf(filler_data,mu1,std1)
median1 = np.median(VsubCO[~np.isnan(VsubCO)])
width1 = np.std(VsubCO[~np.isnan(VsubCO)])
Hist_info_1 = plt.hist(VsubCO[~np.isnan(VsubCO)],bins=50)
#plt.plot(x,f_x1,label="Gaussian Curve")
plt.title("Peak={0},Width={1}".format(round(median1,2),round(width1,2)))
plt.xlabel("H-Alpha Velocity - CO Velocity")
plt.xlim(-100,100)
plt.savefig("HAlpha-CO Histogram.png")

In [ ]:
#mu2, sigma2 = norm.fit(HIsubCO[~np.isnan(HIsubCO)])
#f_x2 = normal_distribution(x,mu2,sigma2) #normal distribution formula
#p1 = norm.pdf(filler_data,mu1,std1)
median2 = np.median(HIsubCO[~np.isnan(HIsubCO)])
width2 = np.std(HIsubCO[~np.isnan(HIsubCO)])
Hist_info_2 = plt.hist(HIsubCO[~np.isnan(HIsubCO)],bins=50,range=(-50,50))
#plt.plot(x,f_x2,label="Gaussian Curve")
plt.title("Peak={0},Width={1}".format(round(median2,2),round(width2,2)))
plt.xlabel("HI Velocity - CO Velocity")
plt.xlim(-100,100)
plt.savefig("HI-CO Histogram.png")

### CC Map Using Subtraction Values

In [ ]:
myDataDF = pd.read_csv("./Redshift CSV/Redshift (incl. CO and HI).csv")
V = np.array(myDataDF["Velocity (km/s)"]) #velocity calculated by me
HI_V = np.array(myDataDF["HI"]) #HI velocity from Amanda's data
CO_V = np.array(myDataDF["CO"]) #CO velocity from Amanda's data
VsubHI = V - HI_V #Halpha - HI
VsubCO = V - CO_V #Halpha - CO
HIsubCO = HI_V - CO_V #HI - CO

In [ ]:
center_RA = (1 + (33/60) + (51.75/3600))*15 #in deg
center_Dec = (30 + (39/60) + (36.630/3600)) # in deg

In [ ]:
def linear_func(x,m,b):
    return (m*x)+b

In [ ]:
def classification_func(RA,Dec):
    
    """
    Parameters
    ----------
    RA: ndarray,req
        Containing RA values used in map
    Dec: ndarray, req
        Containing Dec values used in map
        
    Returns
    ----------
    NorthParabola: ndarray containing bools. True == Inside North parabola
    SouthParabola: ndarray containing bools. True == Inside South parabola
    Center: ndarray containing bools. True == Inside Center region
    """
    
    import cmath
    theta = np.radians(-32)
    
    Dec_Val_NP_List = [] #list containing Dec values along North parabola
    Dec_Val_SP_List = [] #list containing Dec values along South parabola
    
    for RA_Val in RA:
    
        # find a,b,c for quadratic formula
        a_NorthPara = -0.5*np.sin(theta)
        b_NorthPara = 0.5*np.cos(theta)
        c_NorthPara = NorthPara_XCoor - RA_Val
        
        a_SouthPara = 0.5*np.sin(theta)
        b_SouthPara = -0.5*np.cos(theta)
        c_SouthPara = SouthPara_XCoor - RA_Val
        
        # calculate the discriminan
        d_NorthPara = (b_NorthPara**2) - (4*a_NorthPara*c_NorthPara) 
        d_SouthPara = (b_SouthPara**2) - (4*a_SouthPara*c_SouthPara)
        
        t_NorthPara = (-b_NorthPara + cmath.sqrt(d_NorthPara))/(2*a_NorthPara) # find t for North Parabola (Notice the plus)
        t_SouthPara = (-b_SouthPara - cmath.sqrt(d_SouthPara))/(2*a_SouthPara) # find t for South Parabola (Notice the subtract)
        
        Dec_Val_NorthPara = 0.5*(t_NorthPara*np.sin(theta) + (t_NorthPara**2)*np.cos(theta)) + NorthPara_YCoor #find corresponding Dec value using t
        Dec_Val_NP_List.append(Dec_Val_NorthPara)
        
        Dec_Val_SouthPara = -0.5*(t_SouthPara*np.sin(theta) + (t_SouthPara**2)*np.cos(theta)) + SouthPara_YCoor
        Dec_Val_SP_List.append(Dec_Val_SouthPara)
        
    NorthParabola = np.array(Dec-np.array(Dec_Val_NP_List) > 0)
    SouthParabola = np.array(Dec-np.array(Dec_Val_SP_List) < 0)
    Center = []
    for index in range(len(RA)):
        if NorthParabola[index] == False:
            if SouthParabola[index] == False:
                Center.append(True)
            else:
                Center.append(False)
        else:
            Center.append(False)
    
    return NorthParabola,SouthParabola,np.array(Center)

### Halpha - HI

In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("H-Alpha Velocity - HI Velocity",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect('equal')
cc_map = ax.scatter(myDataDF["RA"][~np.isnan(VsubHI)]
                    ,myDataDF["Dec"][~np.isnan(VsubHI)]
                    ,c=VsubHI[~np.isnan(VsubHI)],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
ax.scatter(center_RA,center_Dec,marker="s",color="green",s=40,label="Center")
plt.legend()
plt.grid(True,linestyle="--",linewidth=0.7)

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("H-Alpha - HI.png")

In [ ]:
NorthParabola_HAHI,SouthParabola_HAHI,Center_HAHI = classification_func(myDataDF["RA"][~np.isnan(VsubHI)],myDataDF["Dec"][~np.isnan(VsubHI)])

In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("H-Alpha Velocity - HI Velocity",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect("equal")
plt.grid(True,linestyle="--",linewidth=0.7)

ax.scatter(myDataDF["RA"][~np.isnan(VsubHI)][NorthParabola_HAHI]
                    ,myDataDF["Dec"][~np.isnan(VsubHI)][NorthParabola_HAHI]
                    ,s=7,c="red",label="North")
ax.scatter(myDataDF["RA"][~np.isnan(VsubHI)][SouthParabola_HAHI]
                    ,myDataDF["Dec"][~np.isnan(VsubHI)][SouthParabola_HAHI]
                    ,s=7,c="green",label="South")
ax.scatter(myDataDF["RA"][~np.isnan(VsubHI)][Center_HAHI]
                    ,myDataDF["Dec"][~np.isnan(VsubHI)][Center_HAHI]
                    ,s=7,c="blue",label="Center")
plt.legend()

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("H-Alpha - HI (CC).png")


### Halpha - CO

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("H-Alpha Velocity - CO Velocity",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect('equal')
cc_map = ax.scatter(myDataDF["RA"][~np.isnan(VsubCO)]
                    ,myDataDF["Dec"][~np.isnan(VsubCO)]
                    ,c=VsubCO[~np.isnan(VsubCO)],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
fig.set_figwidth(5)
fig.set_figheight(5)
ax.scatter(center_RA,center_Dec,marker="s",color="green",s=40,label="Center")
plt.legend()
plt.grid(True,linestyle="--",linewidth=0.7)

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

plt.savefig("H-Alpha - CO.png")

In [ ]:
NorthParabola_HACO,SouthParabola_HACO,Center_HACO = classification_func(myDataDF["RA"][~np.isnan(VsubCO)],myDataDF["Dec"][~np.isnan(VsubCO)])


In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("H-Alpha Velocity - CO Velocity",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect("equal")
plt.grid(True,linestyle="--",linewidth=0.7)

ax.scatter(myDataDF["RA"][~np.isnan(VsubCO)][NorthParabola_HACO]
                    ,myDataDF["Dec"][~np.isnan(VsubCO)][NorthParabola_HACO]
                    ,s=7,c="red",label="North")
ax.scatter(myDataDF["RA"][~np.isnan(VsubCO)][SouthParabola_HACO]
                    ,myDataDF["Dec"][~np.isnan(VsubCO)][SouthParabola_HACO]
                    ,s=7,c="green",label="South")
ax.scatter(myDataDF["RA"][~np.isnan(VsubCO)][Center_HACO]
                    ,myDataDF["Dec"][~np.isnan(VsubCO)][Center_HACO]
                    ,s=7,c="blue",label="Center")
plt.legend()

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("H-Alpha - CO (CC).png")


### HI - CO

In [ ]:
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("HI Velocity - CO Velocity",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect('equal')
cc_map = ax.scatter(myDataDF["RA"][~np.isnan(HIsubCO)]
                    ,myDataDF["Dec"][~np.isnan(HIsubCO)]
                    ,c=HIsubCO[~np.isnan(HIsubCO)],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
fig.set_figwidth(5)
fig.set_figheight(5)
ax.scatter(center_RA,center_Dec,marker="s",color="green",s=40,label="Center")
plt.legend()
plt.grid(True,linestyle="--",linewidth=0.7)

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

plt.savefig("HI - CO.png")

In [ ]:
NorthParabola_HICO,SouthParabola_HICO,Center_HICO = classification_func(myDataDF["RA"][~np.isnan(HIsubCO)],myDataDF["Dec"][~np.isnan(HIsubCO)])

In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)",fontsize=12)
ax.set_ylabel("Declination (deg)",fontsize=12)
ax.set_title("HI Velocity - CO Velocity",fontsize=14)
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect("equal")
plt.grid(True,linestyle="--",linewidth=0.7)

ax.scatter(myDataDF["RA"][~np.isnan(HIsubCO)][NorthParabola_HICO]
                    ,myDataDF["Dec"][~np.isnan(HIsubCO)][NorthParabola_HICO]
                    ,s=7,c="red",label="North")
ax.scatter(myDataDF["RA"][~np.isnan(HIsubCO)][SouthParabola_HICO]
                    ,myDataDF["Dec"][~np.isnan(HIsubCO)][SouthParabola_HICO]
                    ,s=7,c="green",label="South")
ax.scatter(myDataDF["RA"][~np.isnan(HIsubCO)][Center_HICO]
                    ,myDataDF["Dec"][~np.isnan(HIsubCO)][Center_HICO]
                    ,s=7,c="blue",label="Center")
plt.legend()

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)
plt.savefig("HI - CO (CC).png")


### Histograms of North, South, and Center Regions

In [ ]:
#Ha-HI
fig, ax = plt.subplots(3)
ax[0].grid(True,linestyle="--",linewidth=0.7)
ax[1].grid(True,linestyle="--",linewidth=0.7)
ax[2].grid(True,linestyle="--",linewidth=0.7)
ax[0].hist(VsubHI[~np.isnan(VsubHI)][NorthParabola_HAHI],bins=40,color="red")
ax[0].set_xlabel("Velocity Difference",fontsize=12)
ax[0].set_title("Ha - HI (North Region)",fontsize=14)
ax[1].hist(VsubHI[~np.isnan(VsubHI)][SouthParabola_HAHI],bins=40,color='green')
ax[1].set_xlabel("Velocity Difference",fontsize=12)
ax[1].set_title("Ha - HI (South Region)",fontsize=14)
ax[2].hist(VsubHI[~np.isnan(VsubHI)][Center_HAHI],bins=40,color='blue')
ax[2].set_xlabel("Velocity Difference",fontsize=12)
ax[2].set_title("Ha - HI (Center Region)",fontsize=14)
fig.set_figwidth(7)
fig.set_figheight(7)
fig.tight_layout()
plt.savefig("HaHI Hist.png")

In [ ]:
#Ha-CO
fig, ax = plt.subplots(3)
ax[0].grid(True,linestyle="--",linewidth=0.7)
ax[1].grid(True,linestyle="--",linewidth=0.7)
ax[2].grid(True,linestyle="--",linewidth=0.7)
ax[0].hist(VsubCO[~np.isnan(VsubCO)][NorthParabola_HACO],bins=40,color="red")
ax[0].set_xlabel("Velocity Difference",fontsize=12)
ax[0].set_title("Ha - CO (North Region)",fontsize=14)
ax[1].hist(VsubCO[~np.isnan(VsubCO)][SouthParabola_HACO],bins=40,color='green')
ax[1].set_xlabel("Velocity Difference",fontsize=12)
ax[1].set_title("Ha - CO (South Region)",fontsize=14)
ax[2].hist(VsubCO[~np.isnan(VsubCO)][Center_HACO],bins=40,color='blue')
ax[2].set_xlabel("Velocity Difference",fontsize=12)
ax[2].set_title("Ha - CO (Center Region)",fontsize=14)
fig.set_figwidth(7)
fig.set_figheight(7)
fig.tight_layout()
plt.savefig("HaCO Hist.png")

In [ ]:
#HI-CO
fig, ax = plt.subplots(3)
ax[0].grid(True,linestyle="--",linewidth=0.7)
ax[1].grid(True,linestyle="--",linewidth=0.7)
ax[2].grid(True,linestyle="--",linewidth=0.7)
ax[0].hist(HIsubCO[~np.isnan(HIsubCO)][NorthParabola_HICO],bins=40,color="red")
ax[0].set_xlabel("Velocity Difference",fontsize=12)
ax[0].set_title("HI - CO (North Region)",fontsize=14)
ax[1].hist(HIsubCO[~np.isnan(HIsubCO)][SouthParabola_HICO],bins=40,color='green')
ax[1].set_xlabel("Velocity Difference",fontsize=12)
ax[1].set_title("HI - CO (South Region)",fontsize=14)
ax[2].hist(HIsubCO[~np.isnan(HIsubCO)][Center_HICO],bins=40,color='blue')
ax[2].set_xlabel("Velocity Difference",fontsize=12)
ax[2].set_title("HI - CO (Center Region)",fontsize=14)
fig.set_figwidth(7)
fig.set_figheight(7)
fig.tight_layout()
plt.savefig("HICO Hist.png")

## Testing For Drag

### $\Delta$ V vs HI V

In [ ]:
HI_V_NoNan = HI_V[~np.isnan(VsubHI)]
VsubHI_NoNan = VsubHI[~np.isnan(VsubHI)]
plt.scatter(HI_V_NoNan,VsubHI_NoNan,s=7)
plt.grid(True,linestyle="--",linewidth=0.7)
plt.title("$\Delta$ V vs HI V",fontsize=14)
plt.xlabel("HI V (km/s)",fontsize=12)
plt.ylabel("$\Delta$ V (km/s)",fontsize=12)
plt.savefig("Delta V vs HI V")

### Position Velocity Diagram

In [ ]:
#HI Data
from astropy.coordinates import Angle
from astropy import units as u
Ri_As,Decli,Maskname,HI,CO,OBJID = read_M33_phot_spec()
Ri_As_NN = Ri_As[~np.isnan(HI)] # Get RA
Decli_NN = Decli[~np.isnan(HI)] # Get Dec
HI_NN = HI[~np.isnan(HI)] # Removing Nan from HI 

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of the Semi-minor axis
m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 

In [ ]:
#Ha data
df1 = pd.read_csv("./Redshift CSV/Redshift.csv")
df1_qop2_sorted = df1["QOP"] >= 2 #use only values that has a QOP of >= 2
V = np.array(df1["Velocity (km/s)"][df1_qop2_sorted]) # Ha velocity
RA = np.array(df1["RA"][df1_qop2_sorted]) # RA of Ha datapoints
DEC = np.array(df1["DEC"][df1_qop2_sorted]) # Dec of Ha datapoints

In [ ]:
def SMA_Projection(V,RA,DEC):
    """
    Parameters:
    ----------
    V: ndarray, req
        Velocity array
    RA: ndarray, req
        Right Ascension array
    DEC: ndarray, req
        Declination array
        
    Returns:
    ----------
    projection: ndarray containing projection along semi-major axis
    """
        
    projection = [] #stores projection along semi-major axis
    
    for index in np.arange(len(V)):
        p = np.array([RA[index],DEC[index]]) # Ra,Dec of data point
        c = np.array([center_RA,center_Dec]) # Center of M33
        #p_c_Vec = p-c # Find vector from center to datapoint

        # Use datapoint to determine the y=mx+b form 
        b = p[1] - (m_SMi*p[0]) #y-intercept of new line

        # Determine the RA and Dec where the new line and the semi-major axis intersect 
        x_inter = (b_SMa-b)/(m_SMi-m_SMa)
        y_inter = linear_func(x_inter,m_SMi,b)

        #p_inter_Vec = p-np.array(x_inter,y_inter) # Find vector from semi-major axis interncept to datapoint
        projection_Vec = c - np.array([x_inter,y_inter]) # Find the vector projection on semi-major axis
        if list(projection_Vec) > [0,0]:
            projection_norm = -np.sqrt(np.sum(projection_Vec**2)) # Determine the norm of projection
        elif list(projection_Vec) < [0,0]:
            projection_norm = np.sqrt(np.sum(projection_Vec**2))
        projection.append(projection_norm)

    return np.array(projection)

In [ ]:
HI_Projection = SMA_Projection(HI_NN,Ri_As_NN,Decli_NN)
Ha_Projection = SMA_Projection(V,RA,DEC)

In [ ]:
fig,ax = plt.subplots(1)
ax.scatter(HI_Projection,HI_NN,s=7,label="HI")
ax.scatter(Ha_Projection,V,s=7,label="H-alpha")
ax.set_xlabel("Projected distance along major axis (degree)",fontsize=12)
ax.set_ylabel("VLSR (km/s)",fontsize=12)
ax.set_title("Position Velocity Diagram",fontsize=14)
ax.grid(True,linestyle="--",linewidth=0.7)
ax.vlines(0.0,-50,-370,linestyle="--")
ax.hlines(-182.1,0.8,-0.8,linestyle="--")
ax.set_ylim(-350,-50)
ax.set_xlim(-0.8,0.8)
ax.legend()
ax.invert_xaxis()
ax.invert_yaxis()
plt.savefig("Position-Velocity Diagram.png")